<a href="https://colab.research.google.com/github/zeynepoykuerdem/GeneticAlgorithminNetflixStock/blob/main/GeneticAlgorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("jainilcoder/netflix-stock-price-prediction")

print(path)

/kaggle/input/netflix-stock-price-prediction


In [ ]:
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.layers import Bidirectional,LSTM, Dense, Input,GRU
from tensorflow.keras.models import Sequential
import numpy as np
import pandas as pd
import kagglehub
import matplotlib.pyplot as plt


def load_data():
  data = pd.read_csv("/kaggle/input/netflix-stock-price-prediction/NFLX.csv")
  data.info()
  return data

load_data()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1009 entries, 0 to 1008
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       1009 non-null   object 
 1   Open       1009 non-null   float64
 2   High       1009 non-null   float64
 3   Low        1009 non-null   float64
 4   Close      1009 non-null   float64
 5   Adj Close  1009 non-null   float64
 6   Volume     1009 non-null   int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 55.3+ KB


,Date,Open,High,Low,Close,Adj Close,Volume
0,2018-02-05,262.000000,267.899994,250.029999,254.259995,254.259995,11896100
1,2018-02-06,247.699997,266.700012,245.000000,265.720001,265.720001,12595800
2,2018-02-07,266.579987,272.450012,264.329987,264.559998,264.559998,8981500
3,2018-02-08,267.079987,267.619995,250.000000,250.100006,250.100006,9306700
4,2018-02-09,253.850006,255.800003,236.110001,249.470001,249.470001,16906900
...,...,...,...,...,...,...,...
1004,2022-01-31,401.970001,427.700012,398.200012,427.140015,427.140015,20047500
1005,2022-02-01,432.959991,458.480011,425.540009,457.130005,457.130005,22542300
1006,2022-02-02,448.250000,451.980011,426.480011,429.480011,429.480011,14346000
1007,2022-02-03,421.440002,429.260010,404.279999,405.600006,405.600006,9905200


python genetic algorithm library --> PyGAD

In [4]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest,f_regression
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
import numpy as np
import pandas as pd
import kagglehub
import matplotlib.pyplot as plt
import random

class GeneticAlgorithm:

  def __init__(self, generation_size, crossover_p, mutation_p, num_of_layers, neurons_per_layer,population_size,selection_type,sequence_lenght):
      self.generation_size = generation_size
      self.crossover_p = crossover_p
      self.mutation_p = mutation_p
      self.num_of_layers = num_of_layers
      self.neurons_per_layer = neurons_per_layer
      self.population_size=population_size
      self.selection_type=selection_type
      self.sequence_lenght=sequence_lenght
      self.new_population=[]

  def load_data(self):
    #loading data
      data = pd.read_csv("/kaggle/input/netflix-stock-price-prediction/NFLX.csv")
      return data

  def pre_processdata(self,data,features,target):
    #normalizing data in order to have same range of data
      scaler_X=MinMaxScaler(feature_range=(0,1))
      scaler_y=MinMaxScaler(feature_range=(0,1))

      X_scaled=scaler_X.fit_transform(data[features])
      y_scaled=scaler_y.fit_transform(data[target])

      return X_scaled,y_scaled

  def create_sequences(self,features,target):
   X,y=[],[]
   for i in range(len(features) - self.sequence_lenght-1):
        X.append(features[i:i+self.sequence_lenght])
        y.append(target[i+self.sequence_lenght])
   X,y=np.array(X),np.array(y)
   return X,y



  def model_builder(self, input_shape,hidden_layers=2):
    model=Sequential()
    model.add(Input(shape=input_shape))
    for i in range(hidden_layers):
      model.add(Dense(self.neurons_per_layer, activation='relu'))

    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    return model


  def train_model(self, X_train, y_train):
        model = self.model_builder(X_train.shape[1:], self.num_of_layers)
        model.fit(X_train, y_train, epochs=3, batch_size=32, verbose=0)
        return model

  def evaluate_model(self, model, X_test, y_test):
        return model.evaluate(X_test, y_test, verbose=0)

  def fitness_function(self, loss_value):
        return 1 / (1 + loss_value)



  def roulette_selection(self, loss_values,population):
    # if selection type is roulette, according to rank scores we select 2 parent with rand(1,sumofrank) in order to get high qualities often

    fitness_values=[self.fitness_function(loss) for loss in loss_values]
    sum_of_fitnesses=np.sum(fitness_values)

    if sum_of_fitnesses==0:
      return random.choice(population).copy()

    pick=random.uniform(0,sum_of_fitnesses)

    current=0
    for i,fscore in enumerate(fitness_values):
      current+=fscore
      if current>=pick:
        return population[i].copy()
    return population[-1].copy()

  def selection_process(self, loss_values,population):
   fitness_values=[self.fitness_function(loss)for loss in loss_values]
   sum_of_fitnesses=np.sum(fitness_values)

   if sum_of_fitnesses==0:
    return random.choice(population).copy()

   if self.selection_type=="Roulette":
    pick=random.uniform(0,sum_of_fitnesses)

    current=0
    for i,fscore in enumerate(fitness_values):
      current+=fscore
      if current>=pick:
        return population[i].copy()
    return population[-1].copy()

   elif self.selection_type=="Tournament":
    torunament_size=3
    tournament_indices=random.sample(range(len(population)),torunament_size)
    winner_indic=max(tournament_indices,key=lambda x:fitness_values[x])
    return population[winner_indic].copy()






  def crossover_process(self,parent1,parent2):
    #one point crossover
    if(random.random()>self.crossover_p):
       return parent1.copy(),parent2.copy()

    # Check if crossover is possible
    if len(parent1) < 2:
        return parent1.copy(), parent2.copy()

    crossover_point=random.randint(1,len(parent1)-1)
    child1=np.concatenate((parent1[:crossover_point],parent2[crossover_point:]))
    child2=np.concatenate((parent2[:crossover_point],parent1[crossover_point:]))
    return child1,child2




  def mutation_process(self,chromosome):
    noise_scale=0.05
    # with the probabilit of cross over, we mutate chromosomes
    if ( random.random()>self.mutation_p):
      return chromosome.copy()

    mutation_point=random.randint(0,len(chromosome)-1)
    chromosome[mutation_point]=1-chromosome[mutation_point]
    return chromosome


#iterating over multiple generations


  def run(self, X_scaled, y_scaled, features):

    X, y = self.create_sequences(X_scaled, y_scaled)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


    num_features=X_train.shape[2]
    chrom_lenght=self.sequence_lenght*num_features
    population = [np.random.rand(chrom_lenght) for _ in range(self.population_size)]

    for generation in range(self.generation_size):
        current_losses = []

        # Evaluate each chromosome
        for chromosome in population:

            model = self.train_model(X_train, y_train)
            loss = self.evaluate_model(model, X_test, y_test)
            current_losses.append(loss)


        best_index = np.argmin(current_losses)
        best_chromosome = population[best_index].copy()
        self.new_population.append(best_chromosome)

        # Selection and generation of new population
        while(len(self.new_population)<self.population_size):
            parent1 = self.selection_process(current_losses, population)
            parent2 = self.selection_process(current_losses, population)
            child1, child2 = self.crossover_process(parent1, parent2)
            child1 = self.mutation_process(child1)
            child2 = self.mutation_process(child2)
            self.new_population.extend([child1, child2])



        population = self.new_population
        print(f"Generation {generation + 1} Best Loss: {current_losses[best_index]}")

    return population

def hyperparameter_printing():
  hyperparameter=[
      'generation_size',
      'crossover_p',
      'mutation_p',
      'selection_type',

  ]
  print(hyperparameter)

def main():

  features = ['Open', 'High', 'Low', 'Volume','Close']
  target = ['Adj Close']



  generation_size=[10,20]
  crossover=[0.7,0.9]
  mutation_probs=[0.01,0.05]
  selection_type=["Roulette","Tournament"]
  for gen in generation_size:
    for cross in crossover:
      for mut in mutation_probs:
          for sel in selection_type:
              ga= GeneticAlgorithm(
                  generation_size=gen,
                  crossover_p=cross,
                  mutation_p=mut,
                  num_of_layers=2,
                  neurons_per_layer=64,
                  population_size=10,
                  selection_type=sel,
                  sequence_lenght=60,
              )
              data = ga.load_data()
              X_scaled, y_scaled= ga.pre_processdata(data, features, target)
              final_population = ga.run(X_scaled, y_scaled, features)
              print("Final generation complete.")
              print(f"Generation size :{gen},Crossover prob :{cross}, Mutation prob:{mut},Selection type:{sel}")
              print("--------------------------------------------------------------------------------")
              print(f"Best cromosome is {ga.new_population[0]}")


